# 目的：任给一个文本资料，判断它出自出自新华社的概率。

In [3]:
from csv import reader
from sys import exit
from math import sqrt
from operator import itemgetter
import numpy as np
import pandas as pd
import jieba
import re
import random
from IPython.display import Image
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

问题解决思路：
1. 先进行数据处理，每篇文章实现为点对 $(x,y)$，其中 $x$ 是文章内容，$y$ 是对应的来源；
2. 这里我们采用逻辑斯蒂模型：是新华社则表为 1，不是则标位 0；
3. 对结果进行评价

In [4]:
news = pd.read_csv('C:/Users/89487/NLP/WEEK_8/sqlResult_1558435.csv', encoding='gb18030')

我们只取出来两个：content 和 source，注意如果是新华社，我们就把他标记为 1，其他的全部标记为 0.

In [48]:
content = pd.read_csv('C:/Users/89487/NLP/WEEK_8/sqlResult_1558435.csv', encoding='gb18030')
content = content.fillna('')

news_content = content['content'].tolist()
# 定义切词函数

def cut(string): return ' '.join(jieba.cut(string))

def token(string):
    return re.findall(r'[\d|\w]+', string)

news_content = [token(n) for n in news_content]

news_content = [' '.join(n) for n in news_content]

news_content = [cut(n) for n in news_content]

In [64]:
news_source = []
for x in news.source:
    if x == '新华社':
        news_source.append(1)
    else:
        news_source.append(0)

In [5]:
news.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [124]:
len(news_source)

89611

In [45]:
len(news_content)

89611

注意：在新闻内容中按照我们的办法分类很不平衡：新华社有 78661 篇，而非新华社只有 10950 篇，为此，我们随机选出 10950 篇新华社样本。

In [52]:
len(news_source)

89611

In [55]:
news_source.count(1)

78661

In [87]:
position_xinhuashe = [i for i in range(len(news_source)) if news_source[i]==1]

In [91]:
random_xinhua = random.sample(position_xinhuashe, 10950)

最后我们需要的样本的位置，一共有 21900 个样本，二者 1:1 的关系：

In [107]:
position = sorted(list(set(random_xinhua)|set(position_not_xinhuashe)))

In [108]:
len(position)

21900

## 下面开始用 Logistics Regression 进行分类

In [112]:
x_train_Chinese, x_test_Chinese, y_train, y_test = train_test_split(news_content_we_use,news_source_we_use,train_size=0.6)

C:\Users\89487\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [113]:
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train_Chinese)

In [109]:
news_content_we_use = [news_content[i] for i in position]
news_source_we_use = [news_source[i] for i in position]

In [115]:
from sklearn import metrics
classifier=LogisticRegression()
classifier.fit(x_train,y_train)
y_tanin_predict=classifier.predict(x_train)
 
print(metrics.classification_report(y_train,y_tanin_predict))       #包含准确率，召回率等信息表
print(metrics.confusion_matrix(y_train,y_tanin_predict))            #混淆矩阵

C:\Users\89487\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6615
           1       1.00      0.93      0.96      6525

   micro avg       0.97      0.97      0.97     13140
   macro avg       0.97      0.97      0.97     13140
weighted avg       0.97      0.97      0.97     13140

[[6585   30]
 [ 425 6100]]


In [116]:
x_demand_prediction=vectorizer.transform(x_test_Chinese)
 
#预测
classifier=LogisticRegression()
classifier.fit(x_train,y_train)
y_predict=classifier.predict(x_demand_prediction)

In [126]:
print(metrics.classification_report(y_test,y_predict))       #包含准确率，召回率等信息表
print(metrics.confusion_matrix(y_test,y_predict))            #混淆矩阵

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4335
           1       0.98      0.89      0.93      4425

   micro avg       0.94      0.94      0.94      8760
   macro avg       0.94      0.94      0.94      8760
weighted avg       0.94      0.94      0.94      8760

[[4273   62]
 [ 486 3939]]


In [120]:
len(y_predict-y_test)

8760

In [123]:
(list(y_predict-y_test)).count(0)

8212

由上面的信息可以看出，直接用 TF-IDF 的效果还不错，注意，因为矩阵的特性我们没有对数据进行预处理。

## 利用感知机进行训练

In [128]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import cross_val_score
ppn = Perceptron(max_iter = 10)
ppn.fit(x_train, y_train)

y_trani_predict_p=ppn.predict(x_train)
 
print(metrics.classification_report(y_train,y_trani_predict_p))       #包含准确率，召回率等信息表
print(metrics.confusion_matrix(y_train,y_trani_predict_p))            #混淆矩阵

C:\Users\89487\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6615
           1       1.00      1.00      1.00      6525

   micro avg       1.00      1.00      1.00     13140
   macro avg       1.00      1.00      1.00     13140
weighted avg       1.00      1.00      1.00     13140

[[6605   10]
 [  20 6505]]


In [129]:
x_demand_prediction=vectorizer.transform(x_test_Chinese)
 
#预测
ppn = Perceptron(max_iter = 10)
ppn.fit(x_train, y_train)
y_predict=classifier.predict(x_demand_prediction)

print(metrics.classification_report(y_test,y_predict))       #包含准确率，召回率等信息表
print(metrics.confusion_matrix(y_test,y_predict))  

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4335
           1       0.98      0.89      0.93      4425

   micro avg       0.94      0.94      0.94      8760
   macro avg       0.94      0.94      0.94      8760
weighted avg       0.94      0.94      0.94      8760

[[4273   62]
 [ 486 3939]]


从效果来看，感知机模型也还是不错。